## Python OOP ile ART1 modellenmesi

In [1]:
# Numpy kütüphanesindeki ihtiyaç duyduğumuz fonksiyonları np. ismiyle çağıracağız.
import numpy as np

# dekoratif çizgiler
dash_line = "\n------------------------------------------------------------\n"
double_line = "\n\n"

In [2]:

class ART1:
    '''ART1 : Adaptif Rezonans Teorisi ile ART1 yapay sinir ağı(YSA) nesnesi oluşturulması
    '''
    def __init__(self, n=5, M=10, rho=0.5):
        '''
        Başlangıç değerlerini ata ve YSA'nın şekil ve sınırlarını belirle
        
        Parametreler:
        -------------
          n     :   Girdinin özellik/feature sayisi [int]
          M     :   Maksimum F2 sınıf sayısı [int]
          rho   :   Benzerlik katsayısı veya vigilance (ihtiyat) parametresi [float]
        '''
        # Karşılaştırma (F1) ve Rekabet (F2) katmanlarına değerler ata
        self.f1 = np.ones(n)
        self.f2 = np.ones(M)
        # İleri (Ai) ve geri besleme (Ag) ağırlıklarını ata
        self.Ai = np.full((n,M) , 1/(1+n) , dtype=np.float)
        self.Ag = np.full((M,n) , 1.0, dtype=np.float)

        # Benzerlik(ihtiyat) katsayısını nesneye nitelik olarak ata 
        self.rho = rho
        # Aktif/mevcut olan sınıfların sayısını hafızada tutmak için parametre
        self.mevcut = 0
        
    def learn(self, X, show=False, predict=False):
        '''ART1 YSA'nı girdi vektörü X ile eğit
            show : verileri goster
        '''
        # İleri yayılım yap
        self.f2 = np.dot(X, self.Ai)

        if show:
            print("Girdi (X): ", dash_line, X, dash_line)
            print("İleri yayılım ağırlık değerleri(Ai): ", dash_line, self.Ai, dash_line)
            print("F2 değerleri (Ai x X): ", dash_line, self.f2, dash_line, double_line)
        # F2 Çıktı değerlerini büyükten küçüğe indeks sıralaması yap & aktif sınıf kadarını filtrele
        EndeksLokasyonları = np.argsort(self.f2.ravel())[::-1]
        print("EndeksLokasyonları: ", EndeksLokasyonları)
        for k in EndeksLokasyonları:
            s = (self.Ag[k,:]*X).sum()/X.sum()
            if s >= self.rho and show:
                print(dash_line, "Kazanan Sınıf/Kategori (k): %s" %k, dash_line)
                print("Geriye doğru ağırlıklar (Ag):",dash_line, self.Ag, dash_line)
                print("Geriye doğru ağırlıklar (kazanan) (Ag_ki): ", dash_line, self.Ag[k,:], dash_line)
                print("Girdi (X): ", dash_line, X, dash_line)
                print("Kazanan sınıf geriye-doğru ağırlık x Girdi(X): () ", dash_line, self.Ag[k,:]*X, dash_line)
                print("s parametresi (s2/s1): ", dash_line, s, dash_line)
            # En büyük değer benzerlik koşulunu sağlıyormu?
            if s >= self.rho:
                if predict==False:
                    # Ağırlıkları güncelle
                    self.Ag[k,:] *= X
                    self.Ai[:,k] = self.Ag[k,:]/(0.5+self.Ag[k,:].sum())
                if show:
                    print("İleri yayılım ağırlık değerleri(Ai): ", dash_line, self.Ai, dash_line)
                    print("Geriye doğru ağırlıklar (Ag):",dash_line, self.Ag, dash_line)
                return self.Ag[k,:], k
            
        # Benzerlik şartı sağlanmıyor...
        # Maksimum sayıda kategori oluşturmamışsak, yeni F2'de node/sınıf/kategori oluştur/aktifleştir
        if self.mevcut < self.f2.size:
            k = self.mevcut
            if predict==False:            
                self.Ag[k,:] *= X
                self.Ai[:,k] = self.Ag[k,:]/(0.5+self.Ag[k,:].sum())
                self.mevcut += 1
            if show:
                print("İleri yayılım ağırlık değerleri(Ai): ", dash_line, self.Ai, dash_line)
                print("Geriye doğru ağırlıklar (Ag):",dash_line, self.Ag, dash_line)
            return self.Ag[:,k], k
        
        #Ağ daha fazla sınıf öğrenemeyirsa yeni sınıf ataması yapma 
        return None, None
        

In [3]:
#1. Örnek:

ART_model = ART1( 3, 3, rho=0.8)
veri = ["101",
        "110",
        "001"]

# Bir örnek için taslak vektör oluştur: [00000]
X = np.zeros(len(veri[0]))

EPOCH = 3
for epoch in range(EPOCH):
    print("-------------------------------------------------------")
    print("Epoch #", epoch+1, ":")
    # Her bir örnek için bir iterasyon ile ağı eğit ve sınıfları yazdır
    for ornek_no in range(len(veri)):
        print(f"*** {ornek_no+1}. iterasyon:\n")
        for ozellik_no in range(len(veri[ornek_no])):
            X[ozellik_no] = (veri[ornek_no][ozellik_no] == '1')
        Ag, k = ART_model.learn(X)
        print("|%s|"%veri[ornek_no],"-> Kategori", k, "\n")
    print("-------------------------------------------------------\n")

-------------------------------------------------------
Epoch # 1 :
*** 1. iterasyon:

EndeksLokasyonları:  [2 1 0]
|101| -> Kategori 2 

*** 2. iterasyon:

EndeksLokasyonları:  [1 0 2]
|110| -> Kategori 1 

*** 3. iterasyon:

EndeksLokasyonları:  [2 0 1]
|001| -> Kategori 2 

-------------------------------------------------------

-------------------------------------------------------
Epoch # 2 :
*** 1. iterasyon:

EndeksLokasyonları:  [2 0 1]
|101| -> Kategori 0 

*** 2. iterasyon:

EndeksLokasyonları:  [1 0 2]
|110| -> Kategori 1 

*** 3. iterasyon:

EndeksLokasyonları:  [2 0 1]
|001| -> Kategori 2 

-------------------------------------------------------

-------------------------------------------------------
Epoch # 3 :
*** 1. iterasyon:

EndeksLokasyonları:  [0 2 1]
|101| -> Kategori 0 

*** 2. iterasyon:

EndeksLokasyonları:  [1 0 2]
|110| -> Kategori 1 

*** 3. iterasyon:

EndeksLokasyonları:  [2 0 1]
|001| -> Kategori 2 

----------------------------------------------------

In [4]:
# Modele reset at
ART_model = ART1( 3, 3, rho=0.8)
EPOCH = 2
for epoch in range(EPOCH):
    print(f"### Epoch {epoch+1} ###")
    # Her bir örnek için bir iterasyon ile ağı eğit ve sınıfları yazdır
    for ornek_no in range(len(veri)):
        print(f"\n*** {ornek_no+1}. iterasyon ***\n")
        for ozellik_no in range(len(veri[ornek_no])):
            X[ozellik_no] = (veri[ornek_no][ozellik_no] == '1')
        Ag, k = ART_model.learn(X, show=False)
        print("|%s|"%veri[ornek_no],"-> Kategori", k, "\n")
    print("************The**End**of**Epoch**#%s************\n\n"%(epoch+1))

### Epoch 1 ###

*** 1. iterasyon ***

EndeksLokasyonları:  [2 1 0]
|101| -> Kategori 2 


*** 2. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|110| -> Kategori 1 


*** 3. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|001| -> Kategori 2 

************The**End**of**Epoch**#1************


### Epoch 2 ###

*** 1. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|101| -> Kategori 0 


*** 2. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|110| -> Kategori 1 


*** 3. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|001| -> Kategori 2 

************The**End**of**Epoch**#2************




In [5]:

#1. Örnek:
ART_model = ART1( 4, 3, rho=0.5)
veri = ["0011",
        "0110",
        "1001",
        "0101",
        "1010"]

# Bir örnek için taslak vektör oluştur: [00000]
X = np.zeros(len(veri[0]))

EPOCH = 3
for epoch in range(EPOCH):
    print(f"### Epoch {epoch+1} ###")
    # Her bir örnek için bir iterasyon ile ağı eğit ve sınıfları yazdır
    for ornek_no in range(len(veri)):
        print(f"\n*** {ornek_no+1}. iterasyon ***\n")
        for ozellik_no in range(len(X)):
            X[ozellik_no] = (veri[ornek_no][ozellik_no] == '1')
        Ag, k = ART_model.learn(X, show=False)
        print("|%s|"%veri[ornek_no],"-> Kategori", k, "\n")
    print("************The**End**of**Epoch**#%s************\n\n"%(epoch+1))
    
    

### Epoch 1 ###

*** 1. iterasyon ***

EndeksLokasyonları:  [2 1 0]
|0011| -> Kategori 2 


*** 2. iterasyon ***

EndeksLokasyonları:  [2 1 0]
|0110| -> Kategori 2 


*** 3. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|1001| -> Kategori 1 


*** 4. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|0101| -> Kategori 1 


*** 5. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|1010| -> Kategori 2 

************The**End**of**Epoch**#1************


### Epoch 2 ###

*** 1. iterasyon ***

EndeksLokasyonları:  [2 1 0]
|0011| -> Kategori 2 


*** 2. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|0110| -> Kategori 2 


*** 3. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|1001| -> Kategori 1 


*** 4. iterasyon ***

EndeksLokasyonları:  [1 0 2]
|0101| -> Kategori 1 


*** 5. iterasyon ***

EndeksLokasyonları:  [2 0 1]
|1010| -> Kategori 2 

************The**End**of**Epoch**#2************


### Epoch 3 ###

*** 1. iterasyon ***

EndeksLokasyonları:  [2 1 0]
|0011| -> Kategori 2 


*** 2. iterasyon ***

Ende

In [7]:
veri2 = ["1111"]

for ornek_no in range(len(veri2)):
        print(f"\n*** {ornek_no+1}. iterasyon ***\n")
        for ozellik_no in range(len(veri2[ornek_no])):
            X[ozellik_no] = (veri2[ornek_no][ozellik_no] == '1')
        Ag, k = ART_model.learn(X, show=False, predict=True)
        print("|%s|"%veri2[ornek_no],"-> Kategori", k, "\n")
        


*** 1. iterasyon ***

EndeksLokasyonları:  [0 2 1]
|1111| -> Kategori 0 



In [8]:
ART_model.Ai

array([[0.2       , 0.        , 0.        ],
       [0.2       , 0.        , 0.        ],
       [0.2       , 0.        , 0.66666667],
       [0.2       , 0.66666667, 0.        ]])

In [9]:
ART_model.Ag

array([[1., 1., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])